In [ ]:
## this agent makes use of configuring the LLM model.
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath('..'))

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langchain_tutorials import classes

SYSTEM_PROMPT = """"You are a warehouse manager who is only allowed to give information concerning the United States.

You have access to two tools:

- get_warehouse_status: use this to get warehouse status for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the warehouse status, make sure they are not in the break room. Use the get_user_location tool to find their location."""

@tool
def get_warehouse_status(loc: str) -> str:
    """Get warehouse status for a given location."""
    return f"The warehouse in {loc} is always up and running and ready to ship today except in the countries France and Canada."


@tool
def get_user_location(runtime: ToolRuntime[classes.RuntimeContext]) -> str:
    """Retrieve information based on user ID."""
    user_id = runtime.context.user_id
    return "main office" if user_id == "1" else "break room"


model = init_chat_model(
    "claude-haiku-4-5-20251001",
    temperature=0.6,
    timeout=10,
    max_tokens=1280
)

agent = create_agent(
    model=model,
    tools=[get_warehouse_status, get_user_location],
    system_prompt=SYSTEM_PROMPT
)

agent.invoke(
    {"messages": [{"role": "user", "content": "What is the status of the warehouse in Milford, PA?"}]},
    context=classes.RuntimeContext(user_id="2"),
)